In [6]:
import pandas as pd
import nltk
from janome.tokenizer import Tokenizer
j_t = Tokenizer()

def tokenizer(text):
    return [tok for tok in j_t.tokenize(text, wakati = True)]

In [7]:
data = pd.read_csv("result_seq2seq.csv")

data

In [8]:
ans = data["answer"]
pred = data["predict"]

In [9]:
def evaluate(path):
    data = pd.read_csv(path)
    ans = data["answer"]
    pred = data["predict"]
    cnt = 0
    # 一致率
    for i in range(len(ans)):
        if ans[i] == pred[i]:
            cnt+=1
    same_percentage = (cnt/len(ans))*100
    # 種類数
    col = []
    for i in range(len(pred)):
        if pred[i] not in col:
            col.append(pred[i])
    ans = len(col)
    # bleu
    df_ans = data.assign(bleu = 0)
    df_ans = df_ans.groupby("input").agg({
    'answer':list,
    "predict": list,
    "bleu" : list
    })
    df_ans = df_ans.reset_index()
    col, ans, pred = [], [], []
    for k in range(len(df_ans)):
        for j in range(len(df_ans["predict"][k])):

            for i in range(len(df_ans["answer"][k])):
                ans_str = df_ans["answer"][k][i]
                ans.append(tokenizer(ans_str))

            #print(k)
            pred = df_ans["predict"][k][j]
            #print(pred)
            pred = tokenizer(pred)
            col.append(nltk.translate.bleu_score.sentence_bleu(ans, pred))
            ave = sum(col)/len(col)
            #print(ave)
            df_ans["bleu"][k][j] = ave
            col, ans = [], []
    input_str, answer, predict, bleu = [], [], [], []
    for i in range(len(df_ans)):
        for j in range(len(df_ans["answer"][i])):
            input_str.append(df_ans["input"][i])
            answer.append(df_ans["answer"][i][j])
            predict.append(df_ans["predict"][i][j])
            bleu.append(df_ans["bleu"][i][j])
            
    df = pd.DataFrame({"input":input_str,"answer":answer,"predict":predict,"bleu":bleu})
    path = path.replace("result", "score")
    df.to_csv(path)
    blue = df["bleu"]
    l = blue.values.tolist()
    ave_blue = sum(l)/len(l)
    return same_percentage,ans, ave_blue  

In [ ]:
evaluate("result_seq2seq.csv")

C:\Users\KEI\anaconda3\envs\school\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\KEI\anaconda3\envs\school\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\KEI\anaconda3\envs\school\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or us

# 完全一致率の計算

In [217]:
cnt = 0
for i in range(len(ans)):
    if ans[i] == pred[i]:
        #print("ans:"+ans[i]+" pred:"+pred[i])
        cnt+=1

In [218]:
cnt

272

In [219]:
same_percentage = (cnt/len(ans))*100

In [220]:
same_percentage

1.2866603595080417

# 種類数の計算

In [221]:
col = []
for i in range(len(pred)):
    if pred[i] not in col:
        col.append(pred[i])

In [222]:
len(col)

75

# BLUEの計算

入力に対する答えが複数あるのですべてに対してスコアを計算する


In [223]:
import nltk

In [224]:
df_ans = data.assign(bleu = 0)

In [225]:
df_ans

,Unnamed: 0,input,answer,predict,bleu
0,457,11時半頃,いらっしゃったうーん,あー,0
1,7866,11時半頃,十一時半に,あー,0
2,2131,15分ぐらいまで,あっ,あですです,0
3,6635,15分ぐらいまで,十五分,あ,0
4,4986,15分ぐらいまで駅に,あはい,そう,0
...,...,...,...,...,...
21135,983,麻雀をするとか,あーあはいはい,あーなんですか,0
21136,9082,麻雀をするとか,あ麻雀はい,あんですか,0
21137,18503,麻雀をするとか,あはい,あ,0
21138,6092,麻雀をするとか,あー麻雀も,あはい,0


In [226]:
df_ans = df_ans.groupby("input").agg({
    'answer':list,
    "predict": list,
    "bleu" : list
})

In [227]:
df_ans = df_ans.reset_index()

In [228]:
df_ans

,input,answer,predict,bleu
0,11時半頃,"[いらっしゃったうーん, 十一時半に]","[あー, あー]","[0, 0]"
1,15分ぐらいまで,"[あっ, 十五分]","[あですです, あ]","[0, 0]"
2,15分ぐらいまで駅に,"[あはい, 15分はい]","[そう, うーん]","[0, 0]"
3,15分ぐらいまで駅に着くまでにかかりました,"[はいだったんですねありがとう, はいうーんうん, ええあそうですねそうだったんですね, あ...","[そう, あえ, あーんですか, あー]","[0, 0, 0, 0]"
4,15分前に,"[あっ, 通りああ, や, 凄いです]","[へえ, あそうなんですか, あー, あーなんですか]","[0, 0, 0, 0]"
...,...,...,...,...
7367,鳥を捕まえるという,"[おー, えー, へー, 捕まえるんですか, ええええ, うん]","[あ, あそうですです, あーです, あそうなんですか, あ, あー]","[0, 0, 0, 0, 0, 0]"
7368,鳥を捕まえるというそんな,"[あはいそうですね, へえ, 鳥を, 鳥, 鳥をですか, 捕まえるんですね, へえ]","[はいありがとうございます嬉しかっふ, あそうなんですか, あそうなんですか, あそうですで...","[0, 0, 0, 0, 0, 0, 0]"
7369,麹町番長の,[そうなんですね],[あーです],[0]
7370,麹町番長の大きなお屋敷のおー石造りのバルコニーみたいな部分だけ残っているとか,"[あー, ああ, はー, そうなんですか]","[そう, あはいです, あです, あ]","[0, 0, 0, 0]"


In [230]:
len(df_ans)

7372

In [231]:
col, ans, pred = [], [], []
for k in range(len(df_ans)):
    for j in range(len(df_ans["predict"][k])):
        
        for i in range(len(df_ans["answer"][k])):
            ans_str = df_ans["answer"][k][i]
            ans.append(tokenizer(ans_str))
        
        #print(k)
        pred = df_ans["predict"][k][j]
        #print(pred)
        pred = tokenizer(pred)
        col.append(nltk.translate.bleu_score.sentence_bleu(ans, pred))
        ave = sum(col)/len(col)
        #print(ave)
        df_ans["bleu"][k][j] = ave
        col, ans = [], []
    

C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

In [232]:
df_ans

,input,answer,predict,bleu
0,11時半頃,"[いらっしゃったうーん, 十一時半に]","[あー, あー]","[0.0, 0.0]"
1,15分ぐらいまで,"[あっ, 十五分]","[あですです, あ]","[0.0, 0.0]"
2,15分ぐらいまで駅に,"[あはい, 15分はい]","[そう, うーん]","[0.0, 0.0]"
3,15分ぐらいまで駅に着くまでにかかりました,"[はいだったんですねありがとう, はいうーんうん, ええあそうですねそうだったんですね, あ...","[そう, あえ, あーんですか, あー]","[2.4655814830110698e-232, 1.1049969584438628e-..."
4,15分前に,"[あっ, 通りああ, や, 凄いです]","[へえ, あそうなんですか, あー, あーなんですか]","[0.0, 1.1640469867513693e-231, 0.0, 1.21833248..."
...,...,...,...,...
7367,鳥を捕まえるという,"[おー, えー, へー, 捕まえるんですか, ええええ, うん]","[あ, あそうですです, あーです, あそうなんですか, あ, あー]","[0.0, 1.2882297539194154e-231, 1.5319719891192..."
7368,鳥を捕まえるというそんな,"[あはいそうですね, へえ, 鳥を, 鳥, 鳥をですか, 捕まえるんですね, へえ]","[はいありがとうございます嬉しかっふ, あそうなんですか, あそうなんですか, あそうですで...","[1.1640469867513693e-231, 1.0719249972567852e-..."
7369,麹町番長の,[そうなんですね],[あーです],[3.418291552750845e-232]
7370,麹町番長の大きなお屋敷のおー石造りのバルコニーみたいな部分だけ残っているとか,"[あー, ああ, はー, そうなんですか]","[そう, あはいです, あです, あ]","[1.821831989445342e-231, 1.384292958842266e-23..."


In [233]:
input_str, answer, predict, bleu = [], [], [], []
for i in range(len(df_ans)):
    for j in range(len(df_ans["answer"][i])):
        input_str.append(df_ans["input"][i])
        answer.append(df_ans["answer"][i][j])
        predict.append(df_ans["predict"][i][j])
        bleu.append(df_ans["bleu"][i][j])

In [234]:
df = pd.DataFrame({"input":input_str,"answer":answer,"predict":predict,"bleu":bleu})

In [235]:
df

,input,answer,predict,bleu
0,11時半頃,いらっしゃったうーん,あー,0.000000e+00
1,11時半頃,十一時半に,あー,0.000000e+00
2,15分ぐらいまで,あっ,あですです,0.000000e+00
3,15分ぐらいまで,十五分,あ,0.000000e+00
4,15分ぐらいまで駅に,あはい,そう,0.000000e+00
...,...,...,...,...
21135,麻雀をするとか,あーあはいはい,あーなんですか,9.283143e-155
21136,麻雀をするとか,あ麻雀はい,あんですか,1.054769e-154
21137,麻雀をするとか,あはい,あ,6.702145e-232
21138,麻雀をするとか,あー麻雀も,あはい,1.491668e-154


In [236]:
df.to_csv('./score_seq2seq.csv')

In [237]:
blue = df["bleu"]

In [238]:
l = blue.values.tolist()

In [239]:
ave_blue = sum(l)/len(l)

In [240]:
ave_blue

0.021538383243622228